In [1]:
import os
import sys
import torch
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from torch import nn

print(torch.__version__)

notebook_dir = os.getcwd()
model_b_root = os.path.abspath(os.path.join(notebook_dir, ".."))
project_root = os.path.abspath(os.path.join(notebook_dir, "../../.."))
print(model_b_root)
print(project_root)

if model_b_root not in sys.path:
    sys.path.append(model_b_root)
    
if project_root not in sys.path:
    sys.path.append(project_root)
    
MODEL_V = "05_simple_transformer"

device = torch.device("mps" if torch.backends.mps.is_available() else "cpu")
device
    

2.2.2
/Users/damianstone/Documents/Code/machine-learning/dl-sepsis-prediction/models/model_B
/Users/damianstone/Documents/Code/machine-learning/dl-sepsis-prediction


device(type='cpu')

In [2]:
from preprocess import preprocess_data

X_train, X_test, y_train, y_test, patient_ids_train, patient_ids_test = preprocess_data(
                                                                            use_last_processed_data=True, 
                                                                            data_file_name="big_imputed_sofa")

Project root: /Users/damianstone/Documents/Code/machine-learning/dl-sepsis-prediction
Loaded from last processed data


In [3]:
X_train.head(5)

,HR,O2Sat,Temp,SBP,MAP,DBP,Resp,EtCO2,BaseExcess,HCO3,...,Hgb,PTT,WBC,Fibrinogen,Platelets,Age,Gender,HospAdmTime,ICULOS,SOFA
126,61.0,98.000000,36.853929,127.0,87.000000,69.995604,15.000000,35.444444,2.395556,25.471622,...,12.622131,39.177551,10.352941,417.166667,239.026786,28.09,1,-0.05,16,4
127,88.0,98.000000,37.280000,140.0,91.153846,69.995604,14.363636,35.444444,2.395556,25.471622,...,12.622131,39.177551,10.352941,417.166667,239.026786,28.09,1,-0.05,28,4
128,65.5,98.000000,36.853929,123.0,91.846154,69.995604,14.090909,35.444444,2.395556,25.471622,...,12.622131,39.177551,10.352941,417.166667,239.026786,28.09,1,-0.05,25,4
129,80.0,97.285714,36.853929,136.0,87.923077,69.995604,15.636364,35.444444,2.395556,25.471622,...,12.622131,39.177551,10.352941,417.166667,239.026786,28.09,1,-0.05,42,4
130,71.0,97.000000,36.853929,144.0,94.666667,69.995604,17.000000,35.444444,2.395556,24.000000,...,14.200000,39.177551,8.100000,417.166667,273.000000,28.09,1,-0.05,6,3


In [4]:
patient_ids_train.head(5)

126    5
127    5
128    5
129    5
130    5
Name: patient_id, dtype: int64

In [5]:
from custom_dataset import SepsisPatientDataset, collate_fn
from torch.utils.data import DataLoader

dataset = SepsisPatientDataset(X_train.values, y_train.values, patient_ids_train.values)
print("First patient records before batching:")
print(dataset[0]) 

batch_size = 32
train_loader = DataLoader(dataset, batch_size=batch_size, shuffle=True, collate_fn=collate_fn)

for padded_X, y_batch, attention_mask in train_loader:
    print("Padded X shape:", padded_X.shape)  # (batch_size, max_seq_len, num_features)
    print("Sample Padded X:", padded_X[0])  # First patient's data
    print("y_batch shape:", y_batch.shape)  # (batch_size,)
    print("Sample y_batch:", y_batch[:5])  # First few labels
    print("Attention mask:", attention_mask[0])  # First patient's mask
    break 

First patient records before batching:
(tensor([[ 6.1000e+01,  9.8000e+01,  3.6854e+01,  ..., -5.0000e-02,
          1.6000e+01,  4.0000e+00],
        [ 8.8000e+01,  9.8000e+01,  3.7280e+01,  ..., -5.0000e-02,
          2.8000e+01,  4.0000e+00],
        [ 6.5500e+01,  9.8000e+01,  3.6854e+01,  ..., -5.0000e-02,
          2.5000e+01,  4.0000e+00],
        ...,
        [ 7.0000e+01,  9.7000e+01,  3.6854e+01,  ..., -5.0000e-02,
          1.8000e+01,  4.0000e+00],
        [ 6.3000e+01,  9.8000e+01,  3.6854e+01,  ..., -5.0000e-02,
          1.1000e+01,  4.0000e+00],
        [ 8.4000e+01,  9.8000e+01,  3.6854e+01,  ..., -5.0000e-02,
          3.0000e+01,  4.0000e+00]]), tensor(0.))
Padded X shape: torch.Size([32, 119, 39])
Sample Padded X: tensor([[ 77.0000,  94.0000,  36.9195,  ...,  -5.8000,  28.0000,   4.0000],
        [110.0000,  95.0000,  36.9195,  ...,  -5.8000,  37.0000,   4.0000],
        [143.0000,  94.0000,  36.2000,  ...,  -5.8000,  34.0000,   6.0000],
        ...,
        [  0.00

In [6]:
X_train.shape[1]

39

In [ ]:
# def adjust_threshold(y_probs, y_batch):
#     best_thresh, best_f1 = 0.5, 0
#     for t in torch.arange(0.1, 0.9, 0.1):  # Test thresholds from 0.1 to 0.9
#         y_preds = (y_probs >= t).float()
#         f1 = 2 * (t_precision(y_preds, y_batch) * t_recall(y_preds, y_batch)) / (t_precision(y_preds, y_batch) + t_recall(y_preds, y_batch) + 1e-8)
#         if f1 > best_f1:
#             best_f1, best_thresh = f1, t
#     return best_thresh

In [8]:
from architectures import TransformerClassifier
from tqdm import tqdm
from torchmetrics import Accuracy, Precision, Recall
from torch import nn

# Initialize model, loss function, optimizer
in_dim = X_train.shape[1] 
valid_heads = [h for h in range(1, in_dim + 1) if in_dim % h == 0]
num_heads = valid_heads[-1] 
print(num_heads)
model = TransformerClassifier(input_dim=in_dim, num_heads=num_heads).to(device)
pos_weight = torch.tensor([2.3], dtype=torch.float32).to(device)
loss_fn = nn.BCEWithLogitsLoss(pos_weight=pos_weight)
optimizer = torch.optim.AdamW(model.parameters(), lr=0.001)

# Training settings
epochs = 100
epoch_counter, loss_counter, acc_counter = [], [], []

# Metrics
t_accuracy = Accuracy(task='binary').to(device)
t_precision = Precision(task='binary').to(device)
t_recall = Recall(task='binary').to(device)

for epoch in range(epochs):
    model.train()
    epoch_loss, epoch_acc, epoch_prec, epoch_rec = 0, 0, 0, 0

    progress_bar = tqdm(train_loader, desc=f"Epoch {epoch+1}/{epochs}", leave=False)

    for X_batch, y_batch, attention_mask in train_loader:
        X_batch, y_batch, attention_mask = (
            X_batch.to(device), 
            y_batch.to(device), 
            attention_mask.to(device)
        )

        # Forward pass with attention mask
        y_logits = model(X_batch, mask=attention_mask)
        y_probs = torch.sigmoid(y_logits)
        y_preds = (y_probs >= 0.5).float()

        # Compute loss
        loss = loss_fn(y_logits.squeeze(), y_batch.float())

        # Compute metrics
        acc = t_accuracy(y_preds.squeeze(), y_batch.float())
        prec = t_precision(y_preds.squeeze(), y_batch.float())
        rec = t_recall(y_preds.squeeze(), y_batch.float())

        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # Update epoch metrics
        epoch_loss += loss.item()
        epoch_acc += acc.item()
        epoch_prec += prec.item()
        epoch_rec += rec.item()

        progress_bar.set_postfix({"Loss": loss.item(), "Acc": acc.item(), "Prec": prec.item(), "Rec": rec.item()})

    # Average epoch metrics
    epoch_loss /= len(train_loader)
    epoch_acc /= len(train_loader)
    epoch_prec /= len(train_loader)
    epoch_rec /= len(train_loader)
    acc_counter.append(epoch_acc)

    print(f"Epoch {epoch+1}/{epochs} | Loss: {epoch_loss:.5f} | Accuracy: {epoch_acc:.2f}% | Precision: {epoch_prec:.2f}% | Recall: {epoch_rec:.2f}%")


39


/Users/damianstone/Documents/Code/machine-learning/dl-sepsis-prediction/venv-sepsis/lib/python3.12/site-packages/torch/nn/modules/transformer.py:286: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.num_heads is odd
  warnings.warn(f"enable_nested_tensor is True, but self.use_nested_tensor is False because {why_not_sparsity_fast_path}")
Epoch 1/100:   0%|          | 0/84 [01:23<?, ?it/s, Loss=1.01, Acc=0.714, Prec=1, Rec=0.2]       

Epoch 1/100 | Loss: 0.98227 | Accuracy: 0.54% | Precision: 0.35% | Recall: 0.52%


Epoch 2/100 | Loss: 0.90504 | Accuracy: 0.64% | Precision: 0.58% | Recall: 0.48%


Epoch 3/100:   0%|          | 0/84 [01:29<?, ?it/s, Loss=1.47, Acc=0.214, Prec=0.154, Rec=1]     

...

Epoch 9/100 | Loss: 0.77815 | Accuracy: 0.75% | Precision: 0.63% | Recall: 0.58%


Epoch 10/100 | Loss: 0.76322 | Accuracy: 0.76% | Precision: 0.62% | Recall: 0.60%


Epoch 11/100:   0%|          | 0/84 [02:10<?, ?it/s, Loss=0.899, Acc=0.714, Prec=0.667, Rec=0.667]

Epoch 11/100 | Loss: 0.75845 | Accuracy: 0.78% | Precision: 0.72% | Recall: 0.55%


KeyboardInterrupt: 